In [54]:
# results are a dataframe with the following columns: Model, Mean LB, Std LB
def compute_mean_std(name_pattern:str) -> pd.DataFrame:
    results_paths = sorted(glob.glob(name_pattern))
    results_dict = {}
    for path in results_paths:
        seed = int(path.split('_')[-1].split('.')[0])
        results_dict[seed] = pd.read_csv(path)
    results = pd.DataFrame()
    for seed in results_dict:
        results = pd.concat([results, results_dict[seed]['LB WPC']], axis=1)
    mean_lb = results.mean(axis=1)
    std_lb = results.std(axis=1)
    results_df = pd.DataFrame({'Model': results_dict[seed]['Model'], 
                               'Mean LB': mean_lb, 
                               'Std LB': std_lb})
    return results_df


In [55]:
def create_latex_table(all_results, all_std, prec=3, determ_tol=1e-3, add_dagger=True):
    """
    Generate a LaTeX table from performance and standard deviation DataFrames.
    
    Args:
        all_results (pd.DataFrame): DataFrame containing the mean performance values.
        all_std (pd.DataFrame): DataFrame containing the standard deviation values.
        prec (int): Number of decimal places for rounding.
        determ_tol (float): Tolerance below which std is considered deterministic.
        add_dagger (bool): Whether to add a dagger symbol to deterministic values.
    
    Returns:
        str: LaTeX formatted table as a string.
    """
    table_rows = []
    
    # Iterate over rows
    for idx, row in all_results.iterrows():
        std_row = all_std.loc[idx]
        row_max = row.iloc[1:].max()  # Find the maximum value in the row, excluding the 'Model' column
        
        formatted_row = [row['Model']]
        
        for col in row.index[1:]:
            mean = f"{row[col]:.{prec}f}"
            std = std_row[col]
            std_is_deterministic = std < determ_tol
            
            if std_is_deterministic:
                formatted_value = f"${mean}$"
                if add_dagger:
                    formatted_value = f"${mean}^\\dagger$"
            else:
                formatted_value = f"${mean} \\pm {round(std, prec)}$"
            
            # Highlight max value
            if row[col] == row_max:
                formatted_value = f"\\mathbf{{{formatted_value.strip('$')}}}"
                formatted_value = f"${formatted_value}$"
            
            formatted_row.append(formatted_value)
        
        table_rows.append(" & ".join(formatted_row) + " \\\\")
    
    # Construct LaTeX table
    columns = " & ".join(all_results.columns)
    latex_table = (
        "\\begin{tabular}{l" + "c" * (len(all_results.columns) - 1) + "}\n"
        "\\hline\n"
        f"{columns} \\\\\n"
        "\\hline\n"
        + "\n".join(table_rows) +
        "\n\\hline\n"
        "\\end{tabular}\n"
    )
    
    return latex_table




In [56]:
# Basic imports
import pandas as pd
import numpy as np
import logging
import warnings
import os
import matplotlib.pyplot as plt
import glob



## Same seed for embeddings, different seeds for sklearn models

In [57]:

sklearn_labelencoder= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_LabelEncoder_seed_*.csv')
print('Sklearn LabelEncoder')
print(sklearn_labelencoder)
print('-------------------\n')

sklearn_onehotencoder= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_OneHotEncoder_seed_*.csv')
print('Sklearn OneHotEncoder')
print(sklearn_onehotencoder)

sklearn_autoint= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_EmbeddingAutoInt_seed_*.csv')
print('Sklearn AutoInt')
print(sklearn_autoint)
print('-------------------\n')

sklearn_category_embedding= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_EmbeddingCategoryEmbedding_seed_*.csv')
print('Sklearn CategoryEmbedding')
print(sklearn_category_embedding)
print('-------------------\n')

sklearn_tabtransformer= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_EmbeddingTabTransformer_seed_*.csv')
print('Sklearn TabTransformer')
print(sklearn_tabtransformer)
print('-------------------\n')



Sklearn LabelEncoder
                          Model   Mean LB        Std LB
0             Linear Regression  0.103923  0.000000e+00
1              Ridge Regression  0.103909  0.000000e+00
2              Lasso Regression  0.115094  0.000000e+00
3         ElasticNet Regression  0.097379  1.462847e-17
4     Bayesian Ridge Regression  0.090467  1.462847e-17
5   Stochastic Gradient Descent -0.033527  9.077060e-02
6                 Decision Tree  0.145827  1.758591e-02
7                 Random Forest  0.247369  1.662151e-02
8                   Extra Trees  0.205042  2.252868e-02
9                      AdaBoost  0.088659  2.339498e-02
10            Gradient Boosting  0.206245  5.112220e-03
11                 XGBRegressor  0.207375  0.000000e+00
12    Support Vector Regression  0.101008  0.000000e+00
13             Gaussian Process  0.112235  0.000000e+00
-------------------

Sklearn OneHotEncoder
                          Model   Mean LB        Std LB
0             Linear Regression  0.26777

In [58]:
## Group all means inside a single dataframe each lb column is an encoding method
all_means = pd.DataFrame()
all_means['Model'] = sklearn_labelencoder['Model']
all_means['LabelEncoder'] = sklearn_labelencoder['Mean LB']
all_means['OneHotEncoder'] = sklearn_onehotencoder['Mean LB']
all_means['AutoInt'] = sklearn_autoint['Mean LB']
all_means['CategoryEmbedding'] = sklearn_category_embedding['Mean LB']
all_means['TabTransformer'] = sklearn_tabtransformer['Mean LB']

all_means


,Model,LabelEncoder,OneHotEncoder,AutoInt,CategoryEmbedding,TabTransformer
0,Linear Regression,0.103923,0.267773,0.239018,0.179289,0.190463
1,Ridge Regression,0.103909,0.256785,0.269739,0.171338,0.210176
2,Lasso Regression,0.115094,0.099321,0.144415,0.134450,0.098821
3,ElasticNet Regression,0.097379,0.112628,0.170785,0.122353,0.105181
4,Bayesian Ridge Regression,0.090467,0.268325,0.261865,0.190347,0.208329
5,Stochastic Gradient Descent,-0.033527,0.267786,0.257334,0.174121,0.213549
6,Decision Tree,0.145827,0.093360,0.113759,0.039225,0.122621
7,Random Forest,0.247369,0.256132,0.268783,0.274259,0.283398
8,Extra Trees,0.205042,0.258021,0.251104,0.258628,0.261496
9,AdaBoost,0.088659,0.109642,0.216977,0.137568,0.170685


In [59]:
## Group all means inside a single dataframe each lb column is an encoding method
all_std = pd.DataFrame()
all_std['Model'] = sklearn_labelencoder['Model']
all_std['LabelEncoder'] = sklearn_labelencoder['Std LB']
all_std['OneHotEncoder'] = sklearn_onehotencoder['Std LB']
all_std['AutoInt'] = sklearn_autoint['Std LB']
all_std['CategoryEmbedding'] = sklearn_category_embedding['Std LB']
all_std['TabTransformer'] = sklearn_tabtransformer['Std LB']

all_std


,Model,LabelEncoder,OneHotEncoder,AutoInt,CategoryEmbedding,TabTransformer
0,Linear Regression,0.000000e+00,0.000000e+00,0.000000e+00,2.925695e-17,2.925695e-17
1,Ridge Regression,0.000000e+00,5.851389e-17,5.851389e-17,0.000000e+00,2.925695e-17
2,Lasso Regression,0.000000e+00,1.462847e-17,0.000000e+00,0.000000e+00,0.000000e+00
3,ElasticNet Regression,1.462847e-17,1.462847e-17,0.000000e+00,1.462847e-17,1.462847e-17
4,Bayesian Ridge Regression,1.462847e-17,0.000000e+00,5.851389e-17,0.000000e+00,2.925695e-17
5,Stochastic Gradient Descent,9.077060e-02,4.238231e-03,4.441117e-03,1.476731e-02,4.299779e-03
6,Decision Tree,1.758591e-02,2.871641e-02,1.781996e-02,1.592702e-02,1.975854e-02
7,Random Forest,1.662151e-02,2.112454e-02,1.977518e-02,1.360479e-02,2.277538e-02
8,Extra Trees,2.252868e-02,1.079611e-02,1.734071e-02,1.649155e-02,3.023973e-02
9,AdaBoost,2.339498e-02,1.785769e-02,2.573109e-02,2.916782e-02,2.795580e-02


In [60]:
# Generate LaTeX code with add_dagger=True
latex_code = create_latex_table(all_means, all_std, prec=3, determ_tol=1e-5, add_dagger=False)
print(latex_code)

\begin{tabular}{lccccc}
\hline
Model & LabelEncoder & OneHotEncoder & AutoInt & CategoryEmbedding & TabTransformer \\
\hline
Linear Regression & $0.104$ & $\mathbf{0.268}$ & $0.239$ & $0.179$ & $0.190$ \\
Ridge Regression & $0.104$ & $0.257$ & $\mathbf{0.270}$ & $0.171$ & $0.210$ \\
Lasso Regression & $0.115$ & $0.099$ & $\mathbf{0.144}$ & $0.134$ & $0.099$ \\
ElasticNet Regression & $0.097$ & $0.113$ & $\mathbf{0.171}$ & $0.122$ & $0.105$ \\
Bayesian Ridge Regression & $0.090$ & $\mathbf{0.268}$ & $0.262$ & $0.190$ & $0.208$ \\
Stochastic Gradient Descent & $-0.034 \pm 0.091$ & $\mathbf{0.268 \pm 0.004}$ & $0.257 \pm 0.004$ & $0.174 \pm 0.015$ & $0.214 \pm 0.004$ \\
Decision Tree & $\mathbf{0.146 \pm 0.018}$ & $0.093 \pm 0.029$ & $0.114 \pm 0.018$ & $0.039 \pm 0.016$ & $0.123 \pm 0.02$ \\
Random Forest & $0.247 \pm 0.017$ & $0.256 \pm 0.021$ & $0.269 \pm 0.02$ & $0.274 \pm 0.014$ & $\mathbf{0.283 \pm 0.023}$ \\
Extra Trees & $0.205 \pm 0.023$ & $0.258 \pm 0.011$ & $0.251 \pm 0.017$ & 

## Same seed for sklearn models, different seeds for embeddings

In [13]:


pt_results = compute_mean_std(name_pattern = '../benchmarks_embed_seeds/pytorch_tabular_seed_*.csv')
print('Pytorch Tabular')
print(pt_results)
print('-------------------\n')

sklearn_labelencoder= compute_mean_std(name_pattern = '../benchmarks_embed_seeds/sklearn_LabelEncoder_seed_*.csv')
print('Sklearn LabelEncoder')
print(sklearn_labelencoder)
print('-------------------\n')

sklearn_onehotencoder= compute_mean_std(name_pattern = '../benchmarks_embed_seeds/sklearn_OneHotEncoder_seed_*.csv')
print('Sklearn OneHotEncoder')
print(sklearn_onehotencoder)

sklearn_autoint= compute_mean_std(name_pattern = '../benchmarks_embed_seeds/sklearn_EmbeddingAutoInt_seed_*.csv')
print('Sklearn AutoInt')
print(sklearn_autoint)
print('-------------------\n')

sklearn_category_embedding= compute_mean_std(name_pattern = '../benchmarks_embed_seeds/sklearn_EmbeddingCategoryEmbedding_seed_*.csv')
print('Sklearn CategoryEmbedding')
print(sklearn_category_embedding)
print('-------------------\n')

sklearn_tabtransformer= compute_mean_std(name_pattern = '../benchmarks_embed_seeds/sklearn_EmbeddingTabTransformer_seed_*.csv')
print('Sklearn TabTransformer')
print(sklearn_tabtransformer)
print('-------------------\n')



Pytorch Tabular
               Model   Mean LB    Std LB
0            AutoInt  0.269537  0.014983
1     TabTransformer  0.224750  0.019862
2  CategoryEmbedding  0.183617  0.031475
-------------------

Sklearn LabelEncoder
                          Model   Mean LB        Std LB
0             Linear Regression  0.103923  0.000000e+00
1              Ridge Regression  0.103909  0.000000e+00
2              Lasso Regression  0.115094  0.000000e+00
3         ElasticNet Regression  0.097379  1.462847e-17
4     Bayesian Ridge Regression  0.090467  1.462847e-17
5   Stochastic Gradient Descent -0.008160  0.000000e+00
6                 Decision Tree  0.152335  2.925695e-17
7                 Random Forest  0.287225  0.000000e+00
8                   Extra Trees  0.189837  0.000000e+00
9                      AdaBoost  0.096175  0.000000e+00
10            Gradient Boosting  0.205103  0.000000e+00
11                 XGBRegressor  0.207375  0.000000e+00
12    Support Vector Regression  0.101008  0.00000

In [14]:
## Group all means inside a single dataframe each lb column is an encoding method
all_means = pd.DataFrame()
all_means['Model'] = sklearn_labelencoder['Model']
all_means['LabelEncoder'] = sklearn_labelencoder['Mean LB']
all_means['OneHotEncoder'] = sklearn_onehotencoder['Mean LB']
all_means['AutoInt'] = sklearn_autoint['Mean LB']
all_means['CategoryEmbedding'] = sklearn_category_embedding['Mean LB']
all_means['TabTransformer'] = sklearn_tabtransformer['Mean LB']

all_means


,Model,LabelEncoder,OneHotEncoder,AutoInt,CategoryEmbedding,TabTransformer
0,Linear Regression,0.103923,0.267773,0.249024,0.218926,0.210931
1,Ridge Regression,0.103909,0.256785,0.259789,0.216753,0.213723
2,Lasso Regression,0.115094,0.099321,0.103340,0.169049,0.098821
3,ElasticNet Regression,0.097379,0.112628,0.156050,0.191007,0.105289
4,Bayesian Ridge Regression,0.090467,0.268325,0.262680,0.216330,0.223163
5,Stochastic Gradient Descent,-0.008160,0.273197,0.259949,0.220773,0.222768
6,Decision Tree,0.152335,0.098165,0.126376,0.118544,0.100746
7,Random Forest,0.287225,0.238286,0.280149,0.286848,0.277669
8,Extra Trees,0.189837,0.272191,0.267412,0.278853,0.265144
9,AdaBoost,0.096175,0.125120,0.193321,0.135008,0.163792


In [15]:
## Group all means inside a single dataframe each lb column is an encoding method
all_std = pd.DataFrame()
all_std['Model'] = sklearn_labelencoder['Model']
all_std['LabelEncoder'] = sklearn_labelencoder['Std LB']
all_std['OneHotEncoder'] = sklearn_onehotencoder['Std LB']
all_std['AutoInt'] = sklearn_autoint['Std LB']
all_std['CategoryEmbedding'] = sklearn_category_embedding['Std LB']
all_std['TabTransformer'] = sklearn_tabtransformer['Std LB']

all_std


,Model,LabelEncoder,OneHotEncoder,AutoInt,CategoryEmbedding,TabTransformer
0,Linear Regression,0.000000e+00,0.000000e+00,0.018398,0.020564,0.021291
1,Ridge Regression,0.000000e+00,5.851389e-17,0.018103,0.024792,0.015596
2,Lasso Regression,0.000000e+00,1.462847e-17,0.017158,0.045572,0.000000
3,ElasticNet Regression,1.462847e-17,1.462847e-17,0.016662,0.043998,0.000226
4,Bayesian Ridge Regression,1.462847e-17,0.000000e+00,0.014563,0.028003,0.022491
5,Stochastic Gradient Descent,0.000000e+00,5.851389e-17,0.016699,0.024401,0.020433
6,Decision Tree,2.925695e-17,0.000000e+00,0.035980,0.057572,0.071597
7,Random Forest,0.000000e+00,2.925695e-17,0.027473,0.022635,0.036845
8,Extra Trees,0.000000e+00,5.851389e-17,0.025590,0.021212,0.035213
9,AdaBoost,0.000000e+00,0.000000e+00,0.034041,0.034022,0.041128


In [16]:
all_std[['LabelEncoder', 'OneHotEncoder', 'AutoInt',
       'CategoryEmbedding', 'TabTransformer']].mean(axis=0)

LabelEncoder         4.179564e-18
OneHotEncoder        1.985293e-17
AutoInt              2.194058e-02
CategoryEmbedding    3.529425e-02
TabTransformer       2.504106e-02
dtype: float64

In [17]:
latex_code = create_latex_table(all_means, all_std, prec=3, determ_tol=1e-5, add_dagger=False)
print(latex_code)

\begin{tabular}{lccccc}
\hline
Model & LabelEncoder & OneHotEncoder & AutoInt & CategoryEmbedding & TabTransformer \\
\hline
Linear Regression & $0.104$ & $\mathbf{0.268}$ & $0.249 \pm 0.018$ & $0.219 \pm 0.021$ & $0.211 \pm 0.021$ \\
Ridge Regression & $0.104$ & $0.257$ & $\mathbf{0.260 \pm 0.018}$ & $0.217 \pm 0.025$ & $0.214 \pm 0.016$ \\
Lasso Regression & $0.115$ & $0.099$ & $0.103 \pm 0.017$ & $\mathbf{0.169 \pm 0.046}$ & $0.099$ \\
ElasticNet Regression & $0.097$ & $0.113$ & $0.156 \pm 0.017$ & $\mathbf{0.191 \pm 0.044}$ & $0.105 \pm 0.0$ \\
Bayesian Ridge Regression & $0.090$ & $\mathbf{0.268}$ & $0.263 \pm 0.015$ & $0.216 \pm 0.028$ & $0.223 \pm 0.022$ \\
Stochastic Gradient Descent & $-0.008$ & $\mathbf{0.273}$ & $0.260 \pm 0.017$ & $0.221 \pm 0.024$ & $0.223 \pm 0.02$ \\
Decision Tree & $\mathbf{0.152}$ & $0.098$ & $0.126 \pm 0.036$ & $0.119 \pm 0.058$ & $0.101 \pm 0.072$ \\
Random Forest & $\mathbf{0.287}$ & $0.238$ & $0.280 \pm 0.027$ & $0.287 \pm 0.023$ & $0.278 \pm 0.037

## Different seeds for embeddings and sklearn models

In [27]:

sklearn_labelencoder= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_LabelEncoder_seed_*.csv')
print('Sklearn LabelEncoder')
print(sklearn_labelencoder)
print('-------------------\n')

sklearn_onehotencoder= compute_mean_std(name_pattern = '../benchmarks_sklearn_seed_0/sklearn_OneHotEncoder_seed_*.csv')
print('Sklearn OneHotEncoder')
print(sklearn_onehotencoder)

sklearn_autoint= compute_mean_std(name_pattern = '../benchmarks_sklearn_embed_seeds/sklearn_EmbeddingAutoInt_seed_*.csv')
print('Sklearn AutoInt')
print(sklearn_autoint)
print('-------------------\n')

sklearn_category_embedding= compute_mean_std(name_pattern = '../benchmarks_sklearn_embed_seeds/sklearn_EmbeddingCategoryEmbedding_seed_*.csv')
print('Sklearn CategoryEmbedding')
print(sklearn_category_embedding)
print('-------------------\n')

sklearn_tabtransformer= compute_mean_std(name_pattern = '../benchmarks_sklearn_embed_seeds/sklearn_EmbeddingTabTransformer_seed_*.csv')
print('Sklearn TabTransformer')
print(sklearn_tabtransformer)
print('-------------------\n')



Sklearn LabelEncoder
                          Model   Mean LB        Std LB
0             Linear Regression  0.103923  0.000000e+00
1              Ridge Regression  0.103909  0.000000e+00
2              Lasso Regression  0.115094  0.000000e+00
3         ElasticNet Regression  0.097379  1.462847e-17
4     Bayesian Ridge Regression  0.090467  1.462847e-17
5   Stochastic Gradient Descent -0.033527  9.077060e-02
6                 Decision Tree  0.145827  1.758591e-02
7                 Random Forest  0.247369  1.662151e-02
8                   Extra Trees  0.205042  2.252868e-02
9                      AdaBoost  0.088659  2.339498e-02
10            Gradient Boosting  0.206245  5.112220e-03
11                 XGBRegressor  0.207375  0.000000e+00
12    Support Vector Regression  0.101008  0.000000e+00
13             Gaussian Process  0.112235  0.000000e+00
-------------------

Sklearn OneHotEncoder
                          Model   Mean LB        Std LB
0             Linear Regression  0.26777

In [30]:
## Group all means inside a single dataframe each lb column is an encoding method
all_means = pd.DataFrame()
all_means['Model'] = sklearn_labelencoder['Model']
all_means['LabelEncoder'] = sklearn_labelencoder['Mean LB']
all_means['OneHotEncoder'] = sklearn_onehotencoder['Mean LB']
all_means['AutoInt'] = sklearn_autoint['Mean LB']
all_means['CategoryEmbedding'] = sklearn_category_embedding['Mean LB']
all_means['TabTransformer'] = sklearn_tabtransformer['Mean LB']

print(all_means)




                          Model  LabelEncoder  OneHotEncoder   AutoInt  \
0             Linear Regression      0.103923       0.267773  0.249024   
1              Ridge Regression      0.103909       0.256785  0.259789   
2              Lasso Regression      0.115094       0.099321  0.103340   
3         ElasticNet Regression      0.097379       0.112628  0.156050   
4     Bayesian Ridge Regression      0.090467       0.268325  0.262680   
5   Stochastic Gradient Descent     -0.033527       0.267786  0.258973   
6                 Decision Tree      0.145827       0.093360  0.113223   
7                 Random Forest      0.247369       0.256132  0.283888   
8                   Extra Trees      0.205042       0.258021  0.269402   
9                      AdaBoost      0.088659       0.109642  0.205905   
10            Gradient Boosting      0.206245       0.256546  0.259334   
11                 XGBRegressor      0.207375       0.236783  0.233387   
12    Support Vector Regression      0

In [31]:
## Group all means inside a single dataframe each lb column is an encoding method
all_std = pd.DataFrame()
all_std['Model'] = sklearn_labelencoder['Model']
all_std['LabelEncoder'] = sklearn_labelencoder['Std LB']
all_std['OneHotEncoder'] = sklearn_onehotencoder['Std LB']
all_std['AutoInt'] = sklearn_autoint['Std LB']
all_std['CategoryEmbedding'] = sklearn_category_embedding['Std LB']
all_std['TabTransformer'] = sklearn_tabtransformer['Std LB']

print(all_std)


                          Model  LabelEncoder  OneHotEncoder   AutoInt  \
0             Linear Regression  0.000000e+00   0.000000e+00  0.018398   
1              Ridge Regression  0.000000e+00   5.851389e-17  0.018103   
2              Lasso Regression  0.000000e+00   1.462847e-17  0.017158   
3         ElasticNet Regression  1.462847e-17   1.462847e-17  0.016662   
4     Bayesian Ridge Regression  1.462847e-17   0.000000e+00  0.014563   
5   Stochastic Gradient Descent  9.077060e-02   4.238231e-03  0.016238   
6                 Decision Tree  1.758591e-02   2.871641e-02  0.050125   
7                 Random Forest  1.662151e-02   2.112454e-02  0.016013   
8                   Extra Trees  2.252868e-02   1.079611e-02  0.024599   
9                      AdaBoost  2.339498e-02   1.785769e-02  0.045035   
10            Gradient Boosting  5.112220e-03   4.695308e-03  0.020701   
11                 XGBRegressor  0.000000e+00   0.000000e+00  0.032879   
12    Support Vector Regression  0.000

In [29]:
latex_code = create_latex_table(all_means, all_std, prec=3, determ_tol=1e-5, add_dagger=False)
print(latex_code)

\begin{tabular}{lccccc}
\hline
Model & LabelEncoder & OneHotEncoder & AutoInt & CategoryEmbedding & TabTransformer \\
\hline
Linear Regression & $0.104$ & $\mathbf{0.268}$ & $0.249 \pm 0.018$ & $0.219 \pm 0.021$ & $0.211 \pm 0.021$ \\
Ridge Regression & $0.104$ & $0.257$ & $\mathbf{0.260 \pm 0.018}$ & $0.217 \pm 0.025$ & $0.214 \pm 0.016$ \\
Lasso Regression & $0.115$ & $0.099$ & $0.103 \pm 0.017$ & $\mathbf{0.169 \pm 0.046}$ & $0.099$ \\
ElasticNet Regression & $0.097$ & $0.113$ & $0.156 \pm 0.017$ & $\mathbf{0.191 \pm 0.044}$ & $0.105 \pm 0.0$ \\
Bayesian Ridge Regression & $0.090$ & $\mathbf{0.268}$ & $0.263 \pm 0.015$ & $0.216 \pm 0.028$ & $0.223 \pm 0.022$ \\
Stochastic Gradient Descent & $-0.034 \pm 0.091$ & $\mathbf{0.268 \pm 0.004}$ & $0.259 \pm 0.016$ & $0.214 \pm 0.029$ & $0.226 \pm 0.02$ \\
Decision Tree & $\mathbf{0.146 \pm 0.018}$ & $0.093 \pm 0.029$ & $0.113 \pm 0.05$ & $0.120 \pm 0.066$ & $0.094 \pm 0.054$ \\
Random Forest & $0.247 \pm 0.017$ & $0.256 \pm 0.021$ & $0.284

In [53]:
# Check percentage improvements
improvements_df = all_means[['Model', 'AutoInt', 'CategoryEmbedding', 'TabTransformer']].copy()

for index in range(len(all_means)):
    max_wpc_classic = all_means.iloc[index, 1:3].max()  # Get the maximum WPC of classic methods (LabelEncoder and OneHotEncoder)
    improvements_df.iloc[index, 1:] = improvements_df.iloc[index, 1:] / max_wpc_classic * 100 -100  # Calculate percentage improvement


print(improvements_df[['AutoInt', 'CategoryEmbedding', 'TabTransformer']].mean(axis=0))  # Print the average percentage improvement for each model

improvements_df = improvements_df.round(2)
print(improvements_df)


AutoInt              17.850047
CategoryEmbedding   -24.951670
TabTransformer       -1.471979
dtype: float64
                          Model  AutoInt  CategoryEmbedding  TabTransformer
0             Linear Regression   -10.74             -33.04          -28.87
1              Ridge Regression     5.04             -33.28          -18.15
2              Lasso Regression    25.48              16.82          -14.14
3         ElasticNet Regression    51.64               8.64           -6.61
4     Bayesian Ridge Regression    -2.41             -29.06          -22.36
5   Stochastic Gradient Descent    -3.90             -34.98          -20.25
6                 Decision Tree   -21.99             -73.10          -15.91
7                 Random Forest     4.94               7.08           10.65
8                   Extra Trees    -2.68               0.24            1.35
9                      AdaBoost    97.90              25.47           55.68
10            Gradient Boosting    -4.53             -25